In [18]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from tqdm import tqdm
import itertools
import datetime
import requests_cache
from IPython.display import HTML

In [7]:
def get_premiere_df():
    url_list = [f"https://i-amabile.com/premier?pageID={i+1}" for i in range(40)]
    response_list = [requests.get(url) for url in tqdm(url_list)]
    soup_list = [BeautifulSoup(response.text) for response in tqdm(response_list)]
    table_list = list(
        itertools.chain.from_iterable(
            soup.select(".single table") for soup in tqdm(soup_list)
        )
    )
    df = pd.DataFrame(dict(table=table_list))
    df["date"] = df.table.apply(
        lambda x: datetime.datetime.strptime(
            x.select("td")[1].text.split("(")[0], "%Y年%m月%d日"
        )
    )
    df["title"] = df.table.apply(lambda x: x.select_one("a").text.lstrip("●"))
    df["link"] = df.table.apply(lambda x: x.select_one("a")["href"])
    df["text"] = df.table.apply(lambda x: x.text)
    df["orchestra"] = df.title.apply(lambda x: x.split("第")[0])
    orchestra_df = df.orchestra.value_counts().to_frame().reset_index().reset_index()
    orchestra_df.columns = ["orchestra_id", "orchestra", "orchestra_count"]
    df = pd.merge(df, orchestra_df)
    df = df.sort_values("date", ascending=False)[
        [
            "orchestra_id",
            "orchestra",
            "orchestra_count",
            "date",
            "title",
            "text",
            "link",
        ]
    ].reset_index(drop=True)
    return df

In [8]:
requests_cache.install_cache('amabile_cache')

In [9]:
df = get_premiere_df()

100%|██████████| 40/40 [00:00<00:00, 487.32it/s]


In [10]:
df.to_csv("amabile_premiere.csv", index=False)

In [16]:
df["link_anchor"] = df.link.apply(lambda url: f'<a href="{url}">link</a>')

In [23]:
HTML(
    df[df.text.str.contains("委嘱")][
        ["orchestra_id", "orchestra", "orchestra_count", "date", "title", "link_anchor"]
    ].to_html(escape=False)
)

,orchestra_id,orchestra,orchestra_count,date,title,link_anchor
7,150,あんさんぶるおとまつコンサート２０１９,1,2019-10-06,あんさんぶるおとまつコンサート２０１９,link
12,8,アンサンブル・フリーEAST,4,2019-09-16,アンサンブル・フリーEAST 第12回演奏会,link
27,125,日髙 剛 20周年 ホルンリサイタル,1,2019-07-13,日髙 剛 20周年 ホルンリサイタル,link
44,254,PRIEM WIND ENSEMBLE,1,2019-05-03,PRIEM WIND ENSEMBLE 第８回演奏会,link
46,85,新宿区吹奏楽団,1,2019-04-21,新宿区吹奏楽団 第30回定期演奏会,link
52,301,甲東ヌーヴェルヴァーグ・ウインドオーケストラ,1,2019-03-10,甲東ヌーヴェルヴァーグ・ウインドオーケストラ 第10回記念定期演奏会,link
60,119,東京大學サクソフォン同好会 10周年記念演奏会,1,2019-01-05,東京大學サクソフォン同好会 10周年記念演奏会,link
62,241,會田瑞樹 ヴィブラフォンリサイタル 〜旅するヴィブラフォン〜,1,2018-12-25,會田瑞樹 ヴィブラフォンリサイタル 〜旅するヴィブラフォン〜,link
65,308,Brass Ensemble Saturday （ブラス・アンサンブル・サタデー）,1,2018-12-15,Brass Ensemble Saturday （ブラス・アンサンブル・サタデー） 第11回定期演奏会 ～古（いにしえ）のしらべ 新たなひびき～,link
87,256,オーケストラアンサンブル・フリーEAST,1,2018-09-02,オーケストラアンサンブル・フリーEAST 第10回演奏会,link
